Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [3]:
#print out data sample

In [4]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [5]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [6]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   23.5s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [7]:
grid_search.best_score_

0.8903150525087514

In [8]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1], dtype=int64)

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [11]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [34]:
y_train = train['ratingCategory']

In [41]:
y_train.head()

0    1
1    0
2    1
3    1
4    1
Name: ratingCategory, dtype: int64

In [33]:
X_train = train['description']

### Define Pipeline Components

In [13]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('rfc', rfc)])

In [ ]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [18]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [19]:
test.head()

,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [42]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'rfc__n_estimators': range(10),
    'rfc__max_depth': (20, 25, 30, 40, 50)
    
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=10, verbose=1)
grid_search.fit(X_train, y_train)

# parameters = {
#     'vect__max_df': ( 0.75, 1.0),
#     'vect__min_df': (.02, .05),
#     'vect__max_features': (500,1000),
#     'clf__n_estimators':(5, 10,),
#     'clf__max_depth':(15,20)
# }

# grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
# grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:   10.9s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:   43.6s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:  1.6min
[Parallel(n_jobs=7)]: Done 600 out of 600 | elapsed:  2.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=7,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'rfc__max_depth': (15, 20, 25), 'rfc__n_estimators': range(10, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [43]:
grid_search.best_score_

0.7264497186200147

In [47]:
grid_search.best_params_

{'rfc__max_depth': 25,
 'rfc__n_estimators': 17,
 'vect__max_df': 0.75,
 'vect__min_df': 0.02}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this compeition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [48]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [49]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [50]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [51]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Challenge

You're trying to achieve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [52]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [53]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

In [54]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [55]:
print(pipe)

Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])


In [56]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.5min


KeyboardInterrupt: 

In [ ]:
grid_search.best_score_

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [58]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()
lsi = Pipeline([('vect', vect), ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('rfc', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [61]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__min_df': (.02, .05),
    'lsi__vect__max_df': (0.75, 1.0),
    'rfc__max_depth':(10,15,20,25,30)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=8, verbose=5)
grid_search.fit(X_train, y_train)

parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   24.9s
[Parallel(n_jobs=8)]: Done 120 out of 120 | elapsed: 17.1min finished
C:\Users\mathi\.conda\envs\U4-S1-NLP\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=8,
       param_grid={'lsi__svd__n_components': [10, 100, 250], 'lsi__vect__max_df': (0.75, 1.0), 'rfc__max_depth': (5, 10, 15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [66]:
grid_search.best_score_

0.7178859799363837

In [67]:
grid_search.best_params_

{'lsi__svd__n_components': 100,
 'lsi__vect__max_df': 0.75,
 'rfc__max_depth': 10}

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
subNumber = 0

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [78]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [79]:
doc = nlp("Two bananas in pyjamas")

In [80]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [175]:
X_train.head()

0    \nSometimes, when whisky is batched, a few lef...
1    \nAn uncommon exclusive bottling of a 6 year o...
2    \nThis release is a port version of Amrut’s In...
3    \nThis 41 year old single cask was aged in a s...
4    \nQuite herbal on the nose, with aromas of dri...
Name: description, dtype: object

In [81]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [111]:
# X = get_word_vectors(data.data)
X = get_word_vectors(X_train)

len(X) == len(X_train)

True

In [112]:
y_train.head()

0    1
1    0
2    1
3    1
4    1
Name: ratingCategory, dtype: int64

In [172]:
train.ratingCategory.value_counts()

1    2881
0    1141
2      65
Name: ratingCategory, dtype: int64

In [113]:
test.head()

,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


In [114]:
rfc.fit(X, y_train)

C:\Users\mathi\.conda\envs\U4-S1-NLP\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [115]:
rfc.score(X, test)

ValueError: Found input variables with inconsistent numbers of samples: [1022, 4087]

## Follow Along

In [72]:
# Apply to your Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from scipy.stats import randint

In [ ]:
# LSI import
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [ ]:
# LSI params # I don't think im using these?
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

In [ ]:
# LSI pipeline
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [ ]:
print(pipe)

In [87]:
# param_dist = {
#     'max_depth' : randint(10, 50),
#     'min_samples_leaf': randint(2, 10)
# }

In [93]:
# Text Extraction & Classification Pipeline

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = GradientBoostingClassifier()
lsi = Pipeline([('vect', vect), ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [118]:
parameters = {
    'lsi__svd__n_components': (10,50,100),
    'lsi__vect__min_df': [.02, .05],
    'lsi__vect__max_df': [0.75, 1.0],
    'clf__max_depth':[10,15,20,25,30],
    'clf__n_estimators': [10, 15, 20]
}
# Try GBC on spacy X

In [99]:
clf.get_params().keys()

dict_keys(['criterion', 'init', 'learning_rate', 'loss', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_iter_no_change', 'presort', 'random_state', 'subsample', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [110]:
X_train.head()

0    \nSometimes, when whisky is batched, a few lef...
1    \nAn uncommon exclusive bottling of a 6 year o...
2    \nThis release is a port version of Amrut’s In...
3    \nThis 41 year old single cask was aged in a s...
4    \nQuite herbal on the nose, with aromas of dri...
Name: description, dtype: object

In [119]:
# tune pipeline with RandomizedSearchCV
search = RandomizedSearchCV(pipe, parameters, n_jobs=6, cv=5, n_iter=15, verbose=5)
search.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   38.6s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:  2.9min
[Parallel(n_jobs=6)]: Done  75 out of  75 | elapsed:  3.4min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=15, n_jobs=6,
          param_distributions={'lsi__svd__n_components': (10, 50, 100), 'lsi__vect__min_df': [0.02, 0.05], 'lsi__vect__max_df': [0.75, 1.0], 'clf__max_depth': [10, 15, 20, 25, 30], 'clf__n_estimators': [10, 15, 20]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=5)

In [120]:
search.best_score_

0.71372644971862

In [121]:
search.best_params_

{'lsi__vect__min_df': 0.02,
 'lsi__vect__max_df': 1.0,
 'lsi__svd__n_components': 50,
 'clf__n_estimators': 10,
 'clf__max_depth': 10}

In [134]:
gbc = GradientBoostingClassifier()

In [173]:
# Using spaCy + GBC + RSCV
param_dist = {                                                        # My best: 72.7%, 73,6%, 74.357%
    'max_depth':[35, 40, 45, 50, 60], # 5, 10, 15, 25                     # My best: 15, x, 
    'n_estimators': [250, 400], #5, 10 , 100, 250                   # My best: 20, x, 
    'min_samples_leaf': randint(4,11), # 2, 3, 5, 8, 13                # My best: 5, x, 
    'max_features': ([.8, 'sqrt', 'log2', None])
}

In [136]:
gbc.get_params().keys()

dict_keys(['criterion', 'init', 'learning_rate', 'loss', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_iter_no_change', 'presort', 'random_state', 'subsample', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [174]:
search = RandomizedSearchCV(gbc, param_dist, n_jobs=7, cv=5, n_iter=15, verbose=10)
search.fit(X, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:   19.2s
[Parallel(n_jobs=7)]: Done  11 tasks      | elapsed:  1.2min
[Parallel(n_jobs=7)]: Done  18 tasks      | elapsed:  3.4min
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:  4.9min
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:  7.0min
[Parallel(n_jobs=7)]: Done  47 tasks      | elapsed: 11.7min
[Parallel(n_jobs=7)]: Done  58 tasks      | elapsed: 13.0min
[Parallel(n_jobs=7)]: Done  70 out of  75 | elapsed: 15.4min remaining:  1.1min
[Parallel(n_jobs=7)]: Done  75 out of  75 | elapsed: 15.6min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=15, n_jobs=7,
          param_distributions={'max_depth': [35, 40, 45, 50, 60], 'n_estimators': [250, 400], 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002377AEEE438>, 'max_features': [0.8, 'sqrt', 'log2', None]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=10)

In [177]:
search.best_score_

0.7428431612429655

In [178]:
pd.DataFrame(search.cv_results_).sort_values('rank_test_score')

C:\Users\mathi\.conda\envs\U4-S1-NLP\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\mathi\.conda\envs\U4-S1-NLP\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\mathi\.conda\envs\U4-S1-NLP\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\mathi\.con

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_n_estimators,params,split0_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
2,15.980880,0.695470,0.033183,0.002984,60,sqrt,9,250,"{'max_depth': 60, 'max_features': 'sqrt', 'min...",0.730159,...,0.742843,0.009410,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
10,136.378947,3.294508,0.027527,0.001493,60,0.8,7,400,"{'max_depth': 60, 'max_features': 0.8, 'min_sa...",0.730159,...,0.740886,0.005876,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,19.143477,0.535632,0.032089,0.002207,60,sqrt,4,400,"{'max_depth': 60, 'max_features': 'sqrt', 'min...",0.730159,...,0.740396,0.007577,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
0,15.187508,0.399392,0.031879,0.002133,35,sqrt,10,400,"{'max_depth': 35, 'max_features': 'sqrt', 'min...",0.727717,...,0.739907,0.009130,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0
11,9.560209,0.297894,0.028534,0.001853,60,log2,6,400,"{'max_depth': 60, 'max_features': 'log2', 'min...",0.732601,...,0.739662,0.005431,5,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,196.547386,5.882335,0.031134,0.003285,60,None,8,400,"{'max_depth': 60, 'max_features': None, 'min_s...",0.722833,...,0.738684,0.011622,6,1.0,1.0,1.0,1.0,1.0,1.0,0.0
13,12.692849,0.489208,0.027726,0.001466,40,sqrt,8,400,"{'max_depth': 40, 'max_features': 'sqrt', 'min...",0.724054,...,0.738439,0.007699,7,1.0,1.0,1.0,1.0,1.0,1.0,0.0
6,141.913926,3.651763,0.028124,0.001596,60,0.8,10,400,"{'max_depth': 60, 'max_features': 0.8, 'min_sa...",0.725275,...,0.737705,0.012040,8,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5,17.242968,0.170710,0.031319,0.002236,45,sqrt,4,250,"{'max_depth': 45, 'max_features': 'sqrt', 'min...",0.735043,...,0.737460,0.002379,9,1.0,1.0,1.0,1.0,1.0,1.0,0.0
12,151.821997,2.631224,0.023257,0.001665,50,None,8,250,"{'max_depth': 50, 'max_features': None, 'min_s...",0.741148,...,0.737216,0.008189,10,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [ ]:
# Consider SGDClassifier + RFC

In [179]:
X_test_spacy = get_word_vectors(test['description'])

In [180]:
# Predictions on test sample
pred = search.predict(X_test_spacy)

In [181]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [182]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [183]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission_spacy_gbc_rscv3.csv', index=False)

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on